In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c7687db15b2a4a164d9c4235d65a9af28858a58615c12d2b8a71c27e630e7278
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's depe

In [ ]:
!pip install -U transformers


In [ ]:
# 🚀 Step 0: Install Dependencies (Run in Colab if needed)
!pip install -q transformers evaluate datasets safetensors

# 🚀 Step 1: Imports
import os
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
import evaluate
import numpy as np

# 🚀 Step 2: Load Dataset
df = pd.read_csv("/content/ProblemSolutionPythonV3.csv").dropna(subset=["Python Code"])
df = df.rename(columns={"Problem": "Query", "Python Code": "Code_Snippet"})
df["Query"] = df["Query"].str.lower()
df["Code_Snippet"] = df["Code_Snippet"].str.lower()

# 🚀 Step 3: Tokenizer
model_checkpoint = "Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 🚀 Step 4: Custom Dataset
class CodeGenDataset(Dataset):
    def __init__(self, queries, codes):
        self.inputs = tokenizer(["generate code: " + q for q in queries],
                                truncation=True, padding="max_length", max_length=256, return_tensors="pt")
        self.targets = tokenizer(codes, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

# 🚀 Step 5: Evaluation Metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_result = bleu.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    em_score = sum([1 if p == l else 0 for p, l in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)

    return {
        "rougeL": round(rouge_result["rougeL"] * 100, 2),
        "bleu": round(bleu_result["bleu"] * 100, 2),
        "exact_match": round(em_score * 100, 2)
    }

# 🚀 Step 6: K-Fold Training + Ensemble
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)
models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    print(f"\n📂 Fold {fold + 1}/{k}")
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

    train_data = df.iloc[train_idx]
    val_data = df.iloc[val_idx]

    train_dataset = CodeGenDataset(train_data["Query"].tolist(), train_data["Code_Snippet"].tolist())
    val_dataset = CodeGenDataset(val_data["Query"].tolist(), val_data["Code_Snippet"].tolist())

    output_dir = f"/content/codet5_fold{fold+1}"
    training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=f"{output_dir}/logs",
    save_total_limit=1,
    logging_steps=500,
    save_steps=500,
    report_to="none"
)


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    models.append(model)

# 🚀 Step 7: Ensemble Saving (First model will be saved as final ensemble checkpoint)
final_model_dir = "/content/codet5_ensemblemm_model"
os.makedirs(final_model_dir, exist_ok=True)
models[0].save_pretrained(final_model_dir, safe_serialization=True)
tokenizer.save_pretrained(final_model_dir)
torch.save(training_args, os.path.join(final_model_dir, "training_args.bin"))
print(f"\n✅ Final ensemble model saved at {final_model_dir}")

# 🚀 Step 8: Ensemble Inference Function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for m in models:
    m.to(device)
    m.eval()

def generate_code(query, max_length=256, num_beams=5):
    input_text = "generate code: " + query.lower()
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).input_ids.to(device)
    attention_mask = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).attention_mask.to(device)

    # Generate outputs from each model
    generated_outputs = []
    with torch.no_grad():
        for m in models:
            generated_ids = m.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=max_length,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1
)

            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            generated_outputs.append(generated_text.strip())

    # Majority vote OR first output (as fallback)
    from collections import Counter
    most_common = Counter(generated_outputs).most_common(1)[0][0]
    return most_common


# 🚀 Step 9: Manual Testing Loop
while True:
    query = input("\n💬 Enter your query (or type 'exit'): ")
    if query.lower() == "exit":
        print("👋 Bye!")
        break
    output = generate_code(query)
    print("\n💡 Generated Code:\n", output)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...


📂 Fold 2/3


Step,Training Loss
500,1.139900
1000,0.764300
1500,0.640000
2000,0.578500
2500,0.518700



📂 Fold 3/3


Step,Training Loss
500,1.147500
1000,0.754800
1500,0.666300
2000,0.580700
2500,0.531400



✅ Final ensemble model saved at /content/codet5_ensemblemm_model

💬 Enter your query (or type 'exit'): python program to perform subtraction of two matrices

💡 Generated Code:
 from numpy import linalg as linalg
  
# taking input of the 1st matrix
print("enter the 1st matrix element:")
print(np.linalg.norm(matrix))
  
# taking input of the 2nd matrix
print("enter the 2nd matrix element:")
print(np.linalg.norm(matrix))
  
# perform subtraction of two matrices
sub_matrix = [[0, 1] for i in range(len(matrix)):
    for j in range(len(matrix[0])):
        sub_matrix[j][i]=matrix[i][j]
  
# display the subtraction of the 2nd matrix
print("subtraction of two matrices is:")
for i in range(len(matrix)):
    print(np.subtract(matrix[i], sub_matrix[i], sub_matrix[i])

💬 Enter your query (or type 'exit'): Python Program to Check Whether a Number is Positive or Negative

💡 Generated Code:
 a=int(input("enter a number: "))
b=int(input("enter a negative number: "))
c=0
while(b!=0):
    if(a%b==0):
 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🔍 Generating predictions:  71%|███████▏  | 2363/3306 [1:42:44<44:19,  2.82s/it]